In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
import concurrent.futures

In [2]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
PAGE_NUM = []
def find_page_num():

    html = getAndParseURL("https://www.zingat.com/karsiyaka-satilik-daire")
    for i in range(1,):
        PAGE_NUM.append("https://www.zingat.com/karsiyaka-satilik-daire" + "?page=" + str(i))
    return PAGE_NUM
find_page_num()

['https://www.zingat.com/karsiyaka-satilik-daire?page=1',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=2',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=3',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=4',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=5',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=6',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=7',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=8',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=9',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=10',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=11',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=12',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=13',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=14',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=15',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=16',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=17',
 'http

In [4]:
ALL_PRODUCT = []
def all_product():
    for i in PAGE_NUM[::]:
        html = getAndParseURL(i)
        for result in html.findAll("a",{"class":"zl-card-inner"}):
            ALL_PRODUCT.append("https://www.zingat.com" + result.get("href"))
    return ALL_PRODUCT
all_product()

['https://www.zingat.com/nese-emlaktan-karsiyaka-imbatlida-satilik-1-1-daire-4957343i',
 'https://www.zingat.com/albatros-7-de-bakimli-tadilatli-manzarali-4-1-4989275i',
 'https://www.zingat.com/karsiyaka-goncalar-mah-girne-bulvari-yakini-satilik-3-1-daire-4926639i',
 'https://www.zingat.com/karsiyaka-bostanlida-satilik-ebeveyn-banyolu-3-1-daire-4955410i',
 'https://www.zingat.com/remax-target-ayse-ayhandan-demirkopru-de-satilik-4-2-dubleks-4857535i',
 'https://www.zingat.com/karsiyaka-bostanli-mah-3-1-satilik-daire-4888168i',
 'https://www.zingat.com/karsiyaka-aksoyda-asansorlu-genis-ve-ferah-satilik-3-1-daire-4984642i',
 'https://www.zingat.com/re-max-targettan-karsiyaka-aksoyda-sifir-daire-4986558i',
 'https://www.zingat.com/karsiyaka-bahriye-ucok-mah-ara-kat-satilik-3-1-daire-4984637i',
 'https://www.zingat.com/semikler-pristina-arkasi-satilik-2-1-daire-4984613i',
 'https://www.zingat.com/keteden-bostanlinin-kalbinde-3-cephe-0-ayarinda-satilik-3-1-4781574i',
 'https://www.zingat.co

In [5]:
RESULT = []
for rslt in ALL_PRODUCT[:]:

    html = getAndParseURL(rslt)
    house_description = html.find("div", {"class":"col-xs-12"}).h1.text
    neighborhood = html.find("div", {"class":"detail-location-path"}).h2.text.replace(",","").strip()
    price = html.find("div", {"class":"col-md-3 price-info-text"}).find("strong",{"itemprop":"price"}).text.replace("TL","").strip()
    net_m2 = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[2].find("span").text.replace("m²","").strip()
    gross_m2 = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[3].find("span").text.replace("m²","").strip()
    room_livingroom = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[4].find("span").text.strip()
    room = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[4].find("span").text.strip()[0]

    RESULT.append([house_description,neighborhood,price,net_m2,gross_m2,room_livingroom,room])
    
columns = ["İLAN_ADI","MAHALLE", "FİYAT","NET","BRÜT","ODA_SALON_SAYISI","ODA_SAYISI",]
df = pd.DataFrame.from_records(RESULT, columns=columns)

In [6]:
df

,İLAN_ADI,MAHALLE,FİYAT,NET,BRÜT,ODA_SALON_SAYISI,ODA_SAYISI
0,Neşe Emlak'tan Karşıyaka İmbatlı'da Satılık 1+...,İmbatlı Karşıyaka\n İzmir,2.200.000,55,65,1+1,1
1,ALBATROS 7 DE BAKIMLI TADİLATLI MANZARALI 4+1,Mavişehir Karşıyaka\n İzmir,9.750.000,184,210,4+1,4
2,Karşıyaka Goncalar Mah Girne Bulvarı Yakını Sa...,Goncalar Karşıyaka\n İzmir,3.750.000,97,110,3+1,3
3,Karşıyaka Bostanlı'da Satılık Ebeveyn Banyolu ...,Bostanlı Karşıyaka\n İzmir,5.950.000,120,130,3+1,3
4,Remax Target Ayşe Ayhandan Demirköprü de satıl...,Şemikler Karşıyaka\n İzmir,8.700.000,145,160,4+2,4
...,...,...,...,...,...,...,...
847,Soyak Siesta Energy'de 1+1 Ara kat Full Eşyalı...,İnönü Karşıyaka\n İzmir,2.550.000,36,52,1+1,1
848,KARŞIYAKA BAHÇELİEVLER'DE 4+1 SATILIK DAİRE,Bahçelievler Karşıyaka\n İzmir,3.950.000,200,220,4+1,4
849,"Karşıyaka Nergiz'de Geniş Sokakta, Dubleks 3+1...",Goncalar Karşıyaka\n İzmir,2.350.000,100,120,3+1,3
850,İzmir Karşıyaka Donanmacı Mah Sahil Yakını Sat...,Donanmacı Karşıyaka\n İzmir,3.050.000,100,120,3+1,3


In [7]:
df.to_csv("zingat_house_price_prediction.csv", encoding="utf-8")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   İLAN_ADI          852 non-null    object
 1   MAHALLE           852 non-null    object
 2   FİYAT             852 non-null    object
 3   NET               852 non-null    object
 4   BRÜT              852 non-null    object
 5   ODA_SALON_SAYISI  852 non-null    object
 6   ODA_SAYISI        852 non-null    object
dtypes: object(7)
memory usage: 46.7+ KB


In [9]:
df.isnull().sum()

İLAN_ADI            0
MAHALLE             0
FİYAT               0
NET                 0
BRÜT                0
ODA_SALON_SAYISI    0
ODA_SAYISI          0
dtype: int64

In [10]:
df["MAHALLE"].unique()

array(['İmbatlı Karşıyaka\n İzmir', 'Mavişehir Karşıyaka\n İzmir',
       'Goncalar Karşıyaka\n İzmir', 'Bostanlı Karşıyaka\n İzmir',
       'Şemikler Karşıyaka\n İzmir', 'Aksoy Karşıyaka\n İzmir',
       'Bahriye Üçok Karşıyaka\n İzmir', 'Donanmacı Karşıyaka\n İzmir',
       'Demirköprü Karşıyaka\n İzmir', 'İnönü Karşıyaka\n İzmir',
       'Bahçelievler Karşıyaka\n İzmir', 'Yalı Karşıyaka\n İzmir',
       'Örnekköy Karşıyaka\n İzmir', 'Atakent Karşıyaka\n İzmir',
       'Alaybey Karşıyaka\n İzmir', 'Nergiz Karşıyaka\n İzmir',
       'Tuna Karşıyaka\n İzmir', 'Fikri Altay Karşıyaka\n İzmir',
       'Tersane Karşıyaka\n İzmir', 'Mustafa Kemal Karşıyaka\n İzmir',
       'Dedebaşı Karşıyaka\n İzmir', 'Bahariye Karşıyaka\n İzmir',
       'Zübeyde Hanım Karşıyaka\n İzmir',
       'Latife Hanım Karşıyaka\n İzmir', 'Cumhuriyet Karşıyaka\n İzmir'],
      dtype=object)